# Travelling salesman problem - A\* algorithm

**Set of cities:** cities are points with coordinates x, y on a plane with height as z coordinate. 

**Moving:** cost of going from city A to city B is equal to the Euclidean distance between two cities, if there exists a road. Two scenarios criteria:
* There are all the direct connections / e.g. 80% of possible connections
* The problem is symmetrical / asymmetrical (if asymmetrical – going up is height +10%, going down: -10%)

**Choosing coordinates:** randomly from the range <-100, 100> for x, y and <0, 50> for z.


In [ ]:
import random
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque

#### **Creating a graph**

In [ ]:
#creating a list of cities
random.seed(12345)
cities = []
num_cities = 10
for i in range (0,num_cities):
  cities.append([i, random.uniform(-100,100), random.uniform(-100,100), random.uniform(0,50)])

#function measuring the distance from point 1 to point 2
def euclidean(point1, point2, sym):

  euclidean = ((point1[1]-point2[1])**2 + (point1[2]-point2[2])**2 + (point1[3]-point2[3])**2)**(1/2)

  if sym == True:
    return euclidean
  elif sym == False:
    if point2[2]>point1[2]:
      return euclidean*1.1
    elif point2[2]<point1[2]:
      return euclidean*0.9
    else:
      return euclidean

#function creating the directed graph, connecting nodes with each other with weighted edges (using euclidean function)
def create_graph(nodes, conn, sym):

  poss_conn_num = len(nodes) * (len(nodes) - 1)   #number of all possible connections

  if conn > 1 or conn <= 0: raise ValueError

  edges = []

  for k in range (0, len(nodes)):
    for i in range (0, len(nodes)):
      if i != k:
        edges.append([nodes[k], nodes[i], euclidean(nodes[k], nodes[i], sym)])

  if conn <= 1 and conn > 0: 
    rem_conn_num = round(poss_conn_num*(1-conn))     #number of paths we need to remove
    for i in range (0, rem_conn_num):
      del edges[random.randrange(0, len(edges))]
  
  return edges

#creating a graph
graph = create_graph(cities, 0.8, False)
#choosing an inicial point of travel
citystart = cities[2][0]

In [ ]:
for edge in graph:
  if edge[0][0] == 8 and edge[1][0] == 3:
    print(edge)

**Task:** Use A\* with inadmissible/admissible heuristics to approximate the solution of travelling salesman problem.

#### **A\* search algorithm**


In [ ]:
def a_star(departure_point):

  start = departure_point  #parent
  A_path = [[start], 0]  #order of the nodes and path length
  no_way_flag = False

  while len(A_path[0])<len(cities)+1:
    visited = set(A_path[0])

    #we try to choose the next node counting the f = g + h value
    #g is the current path length + g_add (means path from parent to the node)
    #h is heuristic, I define it as h = ff * n (min cost value out of all the paths still possible to go through * number of nodes to visit)

    n = num_cities - len(visited)

    best_child = [None, 10000000]  #node number, value of f

    if n != 0:
      #we look for the children
      for path in graph:
        #parent -> child paths
        if path[0][0] == start and path[1][0] not in visited:
          g_add = path[2]
          ff_min = 10000000
          #we search for min of all the possible paths to go through
          for approx_path in graph:
            if (approx_path[0][0] not in visited and approx_path[1][0] not in visited and approx_path[1][0] != path[1][0]) or (approx_path[0][0] not in visited and approx_path[1][0] == departure_point):
              if approx_path[2] < ff_min: 
                ff_min = approx_path[2]
          h = n * ff_min
          f = g_add + h
          if f < best_child[1]: 
            best_child[0] = path[1][0]
            best_child[1] = f
            g_add_best_child = g_add

    else:
      #comeback
      for approx_path in graph:
        if approx_path[0][0] == A_path[0][-1] and approx_path[1][0] == departure_point:
          g_add_best_child = approx_path[2]
          best_child[0] = departure_point
          break
    
    start = best_child[0]  #new parent
    A_path[0].append(start)
    A_path[1] = A_path[1] + g_add_best_child  #actual path cost at the moment

    if A_path[0][-1] == None:
      no_way_flag = True
      break

  if no_way_flag == False: return A_path
  else: return ("No path found")

In [ ]:
a_star(citystart)

[[2, 9, 6, 4, 3, 1, 7, 0, 5, 8, 2], 813.7546418154637]